In [9]:
import numpy as np
import scanpy as sc
import glob
import os
import json
from PIL import Image, ImageDraw
import shapely

In [2]:
data_dir = 'D:/amonell/timecourse_final'
input_folders = glob.glob(os.path.join(data_dir, 'day*'))

##### We are filtering bad parts of the tissue out using a label editor. The following code first exports an image that we can annotate in labelme

In [3]:
for input_file in input_folders:
    print(input_file)
    adata = sc.read(os.path.join(input_file, 'adatas', '03_intial_neighborhoods.h5ad'))
    all_spatial = adata.obsm['X_spatial']
    try:
        del adata.uns['points']
    except:
        print('Points already removed')
    try:
        adata.obs = adata.obs.drop([column for column in adata.obs.columns if 'Topic' in column], axis=1)
    except:
        print('Topics already removed')

    topic_categories = adata.obs['topic'].cat.categories
    topic_colors = adata.uns['topic_colors']
    
    topic_dict = {category: topic_colors[i] for i, category in enumerate(topic_categories)}
    image_colors = [topic_dict.get(v) for v in adata.obs.topic]

    #D6, D7, D30 #downsize = 20
    downsize = 5
    # Example 2D point array (replace this with your own data)
    points = all_spatial/downsize
    
    # Define the size of the image (adjust as needed)
    image_width = 2000
    image_height = 2000
    
    # Create a white canvas as the base image
    base_image = Image.new('RGB', (image_width, image_height), (255, 255, 255))
    
    # Draw the points on the image
    draw = ImageDraw.Draw(base_image)
    point_size = 1  # Size of the points
    ct = 0
    for point in points:
        draw.ellipse((point[0] - point_size, point[1] - point_size, point[0] + point_size, point[1] + point_size),
                     fill=image_colors[ct])
        ct += 1

    try:
        os.mkdir(os.path.join(input_file,'filtering'))
    except:
        print('filtering directory already exists')
    file_path = os.path.join(input_file, 'filtering', 'topic_image.png')
    base_image.save(file_path)
    adata.uns['filtering_downsize'] = downsize
    adata.write(os.path.join(input_file, 'adatas', '03_intial_neighborhoods.h5ad'))

D:/amonell/timecourse_final\day30_SI
Points already removed
filtering directory already exists
D:/amonell/timecourse_final\day6_SI
Points already removed
filtering directory already exists
D:/amonell/timecourse_final\day7_SI_DMSO
Points already removed
filtering directory already exists
D:/amonell/timecourse_final\day7_SI_RARi
Points already removed
filtering directory already exists
D:/amonell/timecourse_final\day8_SI_Ctrl
Points already removed
filtering directory already exists
D:/amonell/timecourse_final\day8_SI_Xcr1DTR
Points already removed
filtering directory already exists
D:/amonell/timecourse_final\day90_SI
Points already removed
filtering directory already exists


#### Go to labelme (pip install labelme > labelme) > open > open topic_image.png > create polygons > Click around the parts of the tissue that need to be removed. 

##### When finished clicking, click enter, name remove, group 0, save to topic_image.json in same directory.

In [19]:
for input_file in input_folders:
    print(input_file)
    adata = sc.read(os.path.join(input_file, 'adatas', '03_intial_neighborhoods.h5ad'))
    json_file_path = os.path.join(input_file, 'filtering', 'topic_image.json')
    all_spatial = adata.obsm['X_spatial']
    
    # Load the JSON data from the file
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)
    
    # Extract relevant information from the JSON data
    image_height = data['imageHeight']
    image_width = data['imageWidth']
    image_path = data['imagePath']
    shapes = data['shapes']
    
    # Process the shapes (annotations)
    removals = []
    points = [] 
    top_points = [] 
    for shape in shapes:
        label = shape['label']
        removals.append(shape['points'])
    
    total_indices = []
    for ir in removals:
        ir_ = np.array(ir)*downsize
        poly = shapely.Polygon(ir_)
        indices = []
        for i in range(len(all_spatial)):
            pt = shapely.Point(all_spatial[i])
            if pt.within(poly):
                indices.append(i)
        total_indices.append(indices)
    
    total_indices=list(set([element for sublist in total_indices for element in sublist]))
    index_set = set(total_indices)
    dont_remove = [i for i in range(len(all_spatial)) if i not in index_set]
    
    adata = adata[dont_remove, :]
    
    adata.write(os.path.join(input_file, 'adatas', '04_tissue_cleared.h5ad'))

D:/amonell/timecourse_final\day30_SI
D:/amonell/timecourse_final\day6_SI
D:/amonell/timecourse_final\day7_SI_DMSO
D:/amonell/timecourse_final\day7_SI_RARi
D:/amonell/timecourse_final\day8_SI_Ctrl
D:/amonell/timecourse_final\day8_SI_Xcr1DTR
D:/amonell/timecourse_final\day90_SI
